In [5]:
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from app import app

import math
import numpy as np
import pandas as pd
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objs as go
import dash as dash
import json
from datetime import datetime as dt
from datetime import timedelta
import re

from modules import module_vobcfault
from modules import module_commLoss
from views.ViewTrainmoveClass import ViewTrainmoveClass
from views.view_commLossListClass import ViewCommLossListClass


import util as util



In [6]:
start_date = dt(2015, 1, 1)
end_date = dt(2016, 1, 1)
start_date, end_date  = util.date2str2(start_date, end_date )

In [18]:
def get_commLoss_by_vobcid_loc(start_date, end_date, velocity_dropdown = None, apstatus = None, commtype = None):
    
    start_date,end_date = util.date2str2(start_date,end_date)
    veld = ''
    apstat = ''
    commT = ''
    if velocity_dropdown is 0: 
        veld = " and velocity = 0"
    if velocity_dropdown is 1: 
        veld = " and NOT(velocity = 0)"
    if apstatus is 1: 
        apstat = " and activePassiveStatus = true"
    if apstatus is 0: 
        apstat = " and activePassiveStatus = false"
    if commtype is not None and commtype != -1:
        commT = " and commType = {}".format(commtype)
 

    query = ("SELECT vobcid, locationName, count(commType) as commLossCount from dlr_vobc_comloss"
                " where commType > 0 and commType < 6 and loggedAt >= '{}' and loggedAt < '{}' {} {} {}" 
                " group by vobcid, locationName"
                " order by commLossCount desc"
                " LIMIT 100").format(start_date, end_date, veld, apstat, commT)

    
    df = util.run_query(query)
    if df is None:
        df = pd.DataFrame() 
    return df


In [19]:
df = pd.DataFrame()
for i in range(5):
    df1 = get_commLoss_by_vobcid_loc(start_date, end_date, None, None, i+1)
    df1["commType"] = i+1
    df = df.append(df1)

In [20]:
df

,vobcid,locationName,commLossCount,commType
0,24,POP-BLA,283,1
1,248,POP-BLA,246,1
2,230,POP-BLA,236,1
3,86,EAI-BLA,208,1
4,240,STR,196,1
...,...,...,...,...
95,274,LIM,344,5
96,178,SHA-BAN,343,5
97,212,LIM,342,5
98,286,LIM,341,5


In [35]:
comLoss = df["commLossCount"].tolist()
comLoss = np.array(comLoss)

In [41]:
fig = go.Figure(data=[go.Scatter3d(
    x=df["vobcid"],
    z=df["locationName"],
    y=df["commType"],
    mode='markers',
    marker=dict(
        size=df["commLossCount"]/max(df["commLossCount"]) * 1000,
        color=df["commType"],               
        #colorscale='Viridis',
        sizemode = 'area',  
        opacity=0.8
    )
)])
fig